In [1]:
from collections import Counter, defaultdict
import numpy as np
import yaml
import json
import pickle
from pyBioInfo.Range import GRange
from pyBioInfo.IO.File import FamFile, FastaFile
from pyBioInfo.Utils import SegmentTools, BundleBuilder, BlockTools, ShiftLoader
import gzip
from collections import defaultdict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import binom

In [2]:
with open("config.yaml") as f:
    samples = yaml.load(f, Loader=yaml.BaseLoader)["samples"]
print(len(samples))

66


In [4]:
chrom0 = "NW_019174257.1"

# 加载NW_019174257.1中的信息，保存为GRange对象
array1 = []
with gzip.open("results/mismatch/merged/all.tsv.gz", "rt") as f:
    for line in f:
        row = line.strip("\n").split("\t")
        # print(chrom)
        chrom = row[0]
        if chrom < chrom0:
            continue
        elif chrom > chrom0:
            break
        else:
            position = int(row[1])
            strand = row[2]
            covs = list(map(int, row[3].split(":")))
            array = []
            for es in row[4].split(":"):
                events = []
                for e in es.split(";"):
                    if e != "":
                        ref, alt, qua = e.split(",")
                        qua = int(qua)
                        events.append([ref, alt, qua])
                array.append(events)
            obj = GRange(chrom=chrom, start=position, end=position + 1, strand=strand)
            obj.coverages = covs
            obj.all_events = array
            array1.append(obj)

In [5]:
len(array1)

252808

In [6]:
for obj1 in array1:
    coverages = obj1.coverages
    all_events = obj1.all_events
    mtypes = []
    for cov, es in zip(coverages, all_events):
        counter = defaultdict(int) # high-confidence mismatch event in single sample
        for ref, alt, qua in es:
            if qua >= 20:
                counter[(ref ,alt)] += 1
        for (ref, alt), num in counter.items():
            ratio = num / cov
            if cov >= 10 and num >= 3 and ratio >= 0.3:
                mtypes.append((ref, alt))
    mtypes = list(set(mtypes)) # 潜在的突变类型
    obj1.high_confidence = mtypes
Counter([len(obj1.high_confidence) for obj1 in array1])

Counter({0: 239326, 1: 13398, 2: 84})

In [67]:
edits = defaultdict(int)
for obj in array1[:10000]:
    if len(obj.snps) > 0:
        continue
        
    ref_base = None
    covs = np.array(obj.coverages)
    counters = []
    for es in obj.all_events:
        counter = defaultdict(int)
        for ref, alt, qua in es:
            ref_base = ref
            if qua >= 0:
                counter[alt] += 1
        counters.append(counter)
    
    for alt in "ACGT":
        if alt == ref_base:
            continue
        counts = np.array([c[alt] for c in counters])
        ratios = counts / covs
        flags = (covs >= 10) & (counts >= 5) & (ratios >= 0)
        if sum(flags) > 0:
            edits[(ref_base, alt)] += 1
            # print(obj.start, obj.strand, ref_base, alt, len(obj.snps), sep="\t")
            # print(counts)
            # print(ratios)
            # print(flags)
            # print(covs[flags])
            # print(counts[flags])
            # print(ratios[flags])
    # break

/home/chenzonggui/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in true_divide
/home/chenzonggui/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in greater_equal


In [68]:
edits, edits[("A", "G")] / sum(edits.values())

(defaultdict(int,
             {('C', 'T'): 88,
              ('A', 'T'): 246,
              ('G', 'A'): 88,
              ('T', 'G'): 67,
              ('T', 'A'): 347,
              ('A', 'G'): 182,
              ('T', 'C'): 157,
              ('C', 'G'): 76,
              ('A', 'C'): 79,
              ('G', 'C'): 45,
              ('G', 'T'): 28,
              ('C', 'A'): 39}),
 0.1262135922330097)

In [51]:
# Load candidate SNPs
array2 = []
with gzip.open("../8_wgs/results/mismatch/merged/all.tsv.gz", "rt") as f:
    for line in f:
        row = line.strip("\n").split("\t")
        chrom = row[0]
        if chrom < "NW_019174257.1":
            continue
        elif chrom > "NW_019174257.1":
            break
        else:
            position = int(row[1])
            covs = list(map(int, row[2].split(":")))
            array = []
            for es in row[3].split(":"):
                events = []
                for e in es.split(";"):
                    if e != "":
                        ref, alt, qua = e.split(",")
                        qua = int(qua)
                        events.append([ref, alt, qua])
                array.append(events)
            obj = GRange(chrom=chrom, start=position, end=position + 1)
            obj.coverages = covs
            obj.all_events = array
            array2.append(obj)

In [52]:
# high confidence SNPs
for obj2 in array2:
    coverages = obj2.coverages
    all_events = obj2.all_events
    mtypes = []
    for cov, es in zip(coverages, all_events):
        counter = defaultdict(int)
        for ref, alt, qua in es:
            if qua >= 20:
                counter[(ref ,alt)] += 1
        for (ref, alt), num in counter.items():
            ratio = num / cov
            if cov >= 10 and num >= 3 and ratio >= 0.3:
                mtypes.append((ref, alt))
    mtypes = list(set(mtypes))
    obj2.high_confidence = mtypes
Counter([len(obj2.high_confidence) for obj2 in array2])

Counter({0: 51408, 1: 78676, 2: 306})

In [55]:
loader = ShiftLoader(array2)
for obj1 in array1:
    obj1.snps = [obj2 for obj2 in loader.fetch(obj=obj1)]
Counter([len(obj1.snps) for obj1 in array1])

Counter({0: 231916, 1: 20892})

In [56]:
array = []
for obj in array1:
    n = 0
    for snp in obj.snps:
        n += len(snp.high_confidence)
    array.append(n)
Counter(array)

Counter({0: 234518, 1: 18179, 2: 111})